In [1]:
import os 
import sys
from glob import glob
import shutil
import time

import cv2
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [38]:
VIDEOS_DIR = "videos"
CAT_DIRS = glob(os.path.join(VIDEOS_DIR, "*"))
CATS = [x.split(os.path.sep)[-1] for x in CAT_DIRS]
NCATS = len(CATS)

IMW = 240
IMH = 135

MODEL_NAME = "weights.hdf5"

msl = 0 # maximum sequence length

In [3]:
for i in range(NCATS):
    category = CATS[i]
    videos = glob(os.path.join(VIDEOS_DIR, category, "*"))
    for video in videos:
        cap = cv2.VideoCapture(video)
        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        msl = max(msl, num_frames)

In [4]:
if os.path.exists("images"):
    shutil.rmtree("images")
os.mkdir("images")

In [5]:
X = []
y = []

In [6]:
for i in range(NCATS):
    category = CATS[i]
    videos = glob(os.path.join(VIDEOS_DIR, category, "*"))
    for j in range(len(videos)):
        video = videos[j]
        count = 0
        cap = cv2.VideoCapture(video)
        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        count = 0
        frame_locations = []
        while count < msl:
            ret, frame = cap.read()
            count += 1
            if not ret:
                frame = np.zeros((IMH, IMW), dtype=float)
            filename = f"{category}_{j:05d}_{count:05d}_{time.time_ns()}.jpg"
            filepath = os.path.join("images", filename)
            cv2.imwrite(filepath, frame)
            frame_locations.append(filepath)
        X.append(frame_locations)
        y.append(category)

In [7]:
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [8]:
X_train_images = []
X_test_images = []

In [9]:
image_loader = lambda x : image.load_img(x, target_size=(IM_WIDTH, IM_HEIGHT, 3))
image_to_array = lambda x : image.img_to_array(x)
image_normalize = lambda x : x / 255.0

for vid_locs in X_train:
    temp = []
    for loc in vid_locs:
        img = cv2.imread(loc)
        img = cv2.resize(img, (IMW, IMH), interpolation=cv2.INTER_AREA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img / 255.0
        temp.append(img)
    X_train_images.append(temp)

for vid_locs in X_test:
    temp = []
    for loc in vid_locs:
        img = cv2.imread(loc)
        img = cv2.resize(img, (IMW, IMH), interpolation=cv2.INTER_AREA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img / 255.0
        temp.append(img)
    X_test_images.append(temp)

In [10]:
X_train_images = np.array(X_train_images)
X_test_images = np.array(X_test_images)

In [11]:
y_train_dummies = pd.get_dummies(y_train)
y_test_dummies = pd.get_dummies(y_test)

In [12]:
X_train_images.shape

(84, 45, 135, 240, 3)

In [16]:
from keras.applications.vgg16 import VGG16

base_model = VGG16(weights="imagenet", include_top=False)

temp_x_train = []
temp_x_test = []

for video_data in X_train_images:
    temp = base_model.predict(video_data)
    temp_x_train.append(temp)

for video_data in X_test_images:
    temp = base_model.predict(video_data)
    temp_x_test.append(temp)

In [24]:
X_train_images = np.array(temp_x_train)
X_test_images = np.array(temp_x_test)

X_train_images.shape

(84, 45, 4, 7, 512)

In [25]:
X_train_images = X_train_images.reshape(X_train_images.shape[0], X_train_images.shape[1], X_train_images.shape[2] * X_train_images.shape[3] * X_train_images.shape[4])
X_test_images = X_test_images.reshape(X_test_images.shape[0], X_test_images.shape[1], X_test_images.shape[2] * X_test_images.shape[3] * X_test_images.shape[4])

In [26]:
X_train_images.shape

(84, 45, 14336)

In [44]:
max_val = max(X_test_images.max(), X_train_images.max())
X_train_images = X_train_images / max_val
X_test_images = X_test_images / max_val

In [45]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

model = Sequential()
model.add(LSTM(256,dropout=0.2, input_shape=(X_train_images.shape[1], X_train_images.shape[2]), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(NCATS, activation="softmax"))

In [46]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 256)               14943232  
                                                                 
 dropout_13 (Dropout)        (None, 256)               0         
                                                                 
 dense_14 (Dense)            (None, 1024)              263168    
                                                                 
 dropout_14 (Dropout)        (None, 1024)              0         
                                                                 
 dense_15 (Dense)            (None, 512)               524800    
                                                                 
 dropout_15 (Dropout)        (None, 512)               0         
                                                                 
 dense_16 (Dense)            (None, 256)              

In [47]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(MODEL_NAME, save_best_only=True, monitor="val_loss", mode="min")
model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])
model.fit(X_train_images, y_train_dummies, epochs=200, validation_data=(
    X_test_images, y_test_dummies), callbacks=[checkpoint], batch_size=128)

Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 1.3826 - accuracy: 0.2619 - val_loss: 1.3714 - val_accuracy: 0.3636
Epoch 2/200
1/1 [==============================] - 1s 709ms/step - loss: 1.4255 - accuracy: 0.3333 - val_loss: 1.3488 - val_accuracy: 0.3636
Epoch 3/200
1/1 [==============================] - 1s 679ms/step - loss: 1.3854 - accuracy: 0.3571 - val_loss: 1.3375 - val_accuracy: 0.3636
Epoch 4/200
1/1 [==============================] - 1s 728ms/step - loss: 1.3570 - accuracy: 0.3571 - val_loss: 1.3290 - val_accuracy: 0.3636
Epoch 5/200
1/1 [==============================] - 1s 851ms/step - loss: 1.3604 - accuracy: 0.2857 - val_loss: 1.3290 - val_accuracy: 0.3182
Epoch 6/200
1/1 [==============================] - 1s 799ms/step - loss: 1.3245 - accuracy: 0.3571 - val_loss: 1.3252 - val_accuracy: 0.3182
Epoch 7/200
1/1 [==============================] - 1s 696ms/step - loss: 1.3031 - accuracy: 0.3571 - val_loss: 1.3205 - val_accuracy: 0.3182
Epoch 8/200
1/1 

In [58]:
y_pred = model.predict(X_test_images)
for i, lis in enumerate(y_pred):
    ind = np.argmax(lis)
    print(f"Actual: {CATS[ind]}\nPredicted: {y_test[i]}\n")

Actual: clear
Predicted: net_shot

Actual: clear
Predicted: clear

Actual: net_shot
Predicted: lift

Actual: smash
Predicted: net_shot

Actual: clear
Predicted: clear

Actual: net_shot
Predicted: lift

Actual: net_shot
Predicted: smash

Actual: net_shot
Predicted: net_shot

Actual: net_shot
Predicted: net_shot

Actual: clear
Predicted: smash

Actual: clear
Predicted: clear

Actual: clear
Predicted: net_shot

Actual: clear
Predicted: clear

Actual: net_shot
Predicted: smash

Actual: clear
Predicted: smash

Actual: clear
Predicted: net_shot

Actual: net_shot
Predicted: smash

Actual: net_shot
Predicted: smash

Actual: net_shot
Predicted: net_shot

Actual: clear
Predicted: lift

Actual: clear
Predicted: net_shot

Actual: net_shot
Predicted: smash

